Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

Создаём сэмпл, дататсет в 1000 значений 


создаем обьекты в датасете

In [2]:
n_samples = 1000

age_owner = np.random.choice(90, n_samples) + 21
length = np.random.choice(120, n_samples) + 15
width = np.random.choice(80, n_samples) + 10

price = length * width * 100 + 126

data = pd.DataFrame({'age_owner': age_owner, 'length': length, 'width': width, 'price': price})
data.head(5)

,age_owner,length,width,price
0,69,99,62,613926
1,43,114,55,627126
2,53,74,85,629126
3,102,134,85,1139126
4,32,16,52,83326


смотрим информацию по дататсету

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age_owner  1000 non-null   int32
 1   length     1000 non-null   int32
 2   width      1000 non-null   int32
 3   price      1000 non-null   int32
dtypes: int32(4)
memory usage: 15.8 KB


Создание описательной статистики.

In [4]:
data.describe()

,age_owner,length,width,price
count,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,65.485000,74.825000,50.426000,3.749012e+05
std,26.057566,33.994117,23.192206,2.480619e+05
min,21.000000,15.000000,10.000000,2.212600e+04
25%,42.000000,48.000000,31.000000,1.682760e+05
50%,65.000000,76.000000,50.000000,3.215760e+05
75%,88.000000,103.000000,71.000000,5.377260e+05
max,110.000000,134.000000,89.000000,1.148526e+06


импортируем библиотеку ошибку

In [6]:
from sklearn.metrics import mean_absolute_error

обучаем модель, не делим на тест и трейн так как нужно посмотреть ошибку которая на обучении, на обобщении ошибка в данном примере не нужна

In [7]:
X = data[['age_owner', 'length', 'width']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

Weights: [  89.36893398 4998.08600439 7266.13744178]
Bias: -371335.15655872383


делаем предсказание и сравниваем ошибку с реальными значениями

In [10]:
pred_values = reg.predict(data[['age_owner', 'length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Error: 58288.522568623725


отбрасываем 1 лишний признак  'age_owner' и пробуем обучить, предсказать, и сравнить ошибку с реальнымы значениями еще раз


In [11]:
X = data[['length', 'width']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['length', 'width']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4999.79774333 7266.11954331]
Bias: -365610.0102358576
Error: 58262.65500021203


Создаем новый признак вручную (проведя аналитику данных и поняв что "price" зависит от "length * width")


In [12]:
data['mult'] = data['length'] * data['width']
data.head(5)

,age_owner,length,width,price,mult
0,69,99,62,613926,6138
1,43,114,55,627126,6270
2,53,74,85,629126,6290
3,102,134,85,1139126,11390
4,32,16,52,83326,832


И строим уже по новому признаку. обучаем, предсказываем, и сраваем ошибку с реальнымы значениями еще раз


In [13]:
X = data[['mult']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [100.]
Bias: 126.00000000011642
Error: 5.677429726347327e-11


Тепрь импортируем библиотеку для автоматизации процесса выше

In [14]:
from sklearn.preprocessing import PolynomialFeatures

используем сторую степень полинома 

и генерируем новые признаки


In [15]:
poly = PolynomialFeatures(2)

X = poly.fit_transform(data[['age_owner','length','width']])
y = data['price']

poly.get_feature_names()

['1', 'x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2', 'x2^2']

 обучаем, предсказываем, и сраваем ошибку с реальнымы значениями еще раз

In [16]:
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.00000000e+00 -7.59570185e-12 -1.93578487e-12 -2.63755684e-12
  4.92939023e-14  5.32907052e-14 -3.55271368e-14  0.00000000e+00
  1.00000000e+02  8.52651283e-14]
Bias: 126.0
Error: 1.7575439414940774e-10


когда модель обучалась она нашла признак 'x1 x2' из новых который хорошо соответсвует таргетной переменной 'price' и придала ему соответствующий вес 1.00000000e+02, все остальные признаки имеют незначительный вес.